We already *know* what the shape of each particle's curve given a q and fres should look like, it's just 1/ f-fres^2 - just sum each contribution, envelope, with basinhopper

Relaxation time might itself change with freq.